# Fit an H2O GBM on the Lending Club data

### Imports

In [1]:
import pandas as pd
import numpy as np
import random, time, os, pickle

import matplotlib.pyplot as plt
from feature_engine import categorical_encoders as ce
from feature_engine import discretisers as dsc
from feature_engine import missing_data_imputers as mdi
from feature_engine import feature_selection as fs
from sklearn.pipeline import Pipeline as pipe

import h2o
from h2o.estimators import H2OXGBoostEstimator, H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

%matplotlib inline

### Read data into pandas and fix date columns

In [2]:
df = pd.read_csv("../data/lending_club_loan_two.csv")

In [3]:
## Fix date columns:
dt_cols = ['issue_d','earliest_cr_line']

df.loc[:,dt_cols] = df.loc[:,dt_cols] \
  .applymap(lambda x: np.nan if x in ['null','NULL',''] else x) \
  .apply(lambda x: pd.to_datetime(x,format = "%b-%Y"))

### Split into train and test

In [4]:
p_trn = 0.8
n = df.shape[0]

df = df.sample(frac=1,random_state=1).reset_index(drop=True)

df_train = df.iloc[:int(n*p_trn),:].copy()
df_test = df.iloc[int(n*p_trn):,:].copy()

In [5]:
df_train.head(5)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,8000.0,36 months,16.20,282.05,C,C4,Counselor,7 years,RENT,50000.0,Verified,2013-10-01,Fully Paid,debt_consolidation,Consolidation loan,17.64,2000-11-01,13.0,1.0,9034.0,68.4,30.0,w,INDIVIDUAL,0.0,1.0,Unit 9587 Box 8878\r\nDPO AA 29597
1,9000.0,36 months,14.33,309.05,C,C1,American Portfolios,4 years,RENT,40000.0,Not Verified,2012-09-01,Fully Paid,credit_card,CC Refinance Loan,15.87,1980-05-01,13.0,0.0,14593.0,71.2,30.0,f,INDIVIDUAL,0.0,0.0,"314 Mercer Route\r\nWest Kathryn, TX 48052"
2,12000.0,60 months,12.49,269.92,B,B5,Senior Analyst,4 years,MORTGAGE,73000.0,Not Verified,2014-07-01,Fully Paid,credit_card,Credit card refinancing,28.63,2001-08-01,11.0,0.0,18633.0,62.1,60.0,w,INDIVIDUAL,2.0,0.0,"5389 Brandi Springs\r\nAshleymouth, LA 29597"
3,20000.0,36 months,6.62,614.08,A,A2,QA Manager,9 years,MORTGAGE,110000.0,Source Verified,2013-12-01,Fully Paid,debt_consolidation,Debt Freedom,15.45,1993-10-01,8.0,0.0,5275.0,38.2,24.0,f,INDIVIDUAL,2.0,0.0,"166 Lopez Key Apt. 864\r\nSouth Angelaland, CO..."
4,13600.0,36 months,7.29,421.74,A,A4,Caxton Associates,7 years,MORTGAGE,115000.0,Source Verified,2011-02-01,Fully Paid,home_improvement,Home Improvement Loan,6.84,1996-10-01,9.0,0.0,191.0,0.8,17.0,f,INDIVIDUAL,NaN,0.0,48033 Xavier Squares Suite 893\r\nSouth Rubenb...


## Verify data types

### Make sure all columns are of the correct data type

In [6]:
dtype_dict = df_train.dtypes.to_dict()

In [7]:
numeric_cols = [k for k,v in dtype_dict.items() if pd.api.types.is_numeric_dtype(v)]
numeric_cols.sort()

print("Numeric columns: " + ", ".join(numeric_cols))

Numeric columns: annual_inc, dti, installment, int_rate, loan_amnt, mort_acc, open_acc, pub_rec, pub_rec_bankruptcies, revol_bal, revol_util, total_acc


In [8]:
object_cols = [k for k,v in dtype_dict.items() if pd.api.types.is_object_dtype(v)]
object_cols.sort()

print("Object columns: " + ", ".join(object_cols))

Object columns: address, application_type, emp_length, emp_title, grade, home_ownership, initial_list_status, loan_status, purpose, sub_grade, term, title, verification_status


In [9]:
datetime_cols = [k for k,v in dtype_dict.items() if pd.api.types.is_datetime64_any_dtype(v)]
datetime_cols.sort()

print("Datetime columns: " + ", ".join(datetime_cols))

Datetime columns: earliest_cr_line, issue_d


In [10]:
other_cols = [c for c in df.columns if c not in numeric_cols + object_cols + datetime_cols]
print("Columns not accounted for: " + ", ".join(other_cols))

Columns not accounted for: 


## Apply NULL/null/NA/NaN consistently

Different datesources may result in different formats for null/missing values. It's typically a good idea to apply a consistent format. I'll do this by replacing 'NULL', 'null' and '' in character columns with `np.nan`.

In [11]:
df_train.loc[:,object_cols] = df_train.loc[:,object_cols] \
  .applymap(lambda x: np.nan if str(x).lower() in ['null',''] else x)

In [12]:
target = 'loan_status'

object_cols_x_target = [c for c in object_cols if c != target]

cols_to_drop = datetime_cols + ['address']

p = pipe([
    # Add missing indicators for numeric features
    ("num_nan_ind",mdi.AddMissingIndicator(variables=numeric_cols)),
    # Add missing level for categorical features
    ("fill_cat_nas",mdi.CategoricalVariableImputer(
        fill_value = "_MISSING_", variables=object_cols_x_target)),
    # Bin rare levels of categorical variables
    ("rare_cats",ce.RareLabelCategoricalEncoder(
        tol = 0.02, replace_with = "_RARE_", variables=object_cols_x_target)),
    # Impute missing numeric variables with media
    ("rmmean",mdi.MeanMedianImputer(imputation_method = 'median',variables=numeric_cols)),
    # Drop dates and address columns
    ("drop_date",fs.DropFeatures(features_to_drop=cols_to_drop))])

In [13]:
df_train_prepped = p.fit_transform(df_train)

In [14]:
df_train_prepped.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,mort_acc_na,pub_rec_bankruptcies_na,revol_util_na
0,8000.0,36 months,16.20,282.05,C,C4,_RARE_,7 years,RENT,50000.0,Verified,Fully Paid,debt_consolidation,_RARE_,17.64,13.0,1.0,9034.0,68.4,30.0,w,INDIVIDUAL,0.0,1.0,0,0,0
1,9000.0,36 months,14.33,309.05,C,C1,_RARE_,4 years,RENT,40000.0,Not Verified,Fully Paid,credit_card,_RARE_,15.87,13.0,0.0,14593.0,71.2,30.0,f,INDIVIDUAL,0.0,0.0,0,0,0
2,12000.0,60 months,12.49,269.92,B,B5,_RARE_,4 years,MORTGAGE,73000.0,Not Verified,Fully Paid,credit_card,Credit card refinancing,28.63,11.0,0.0,18633.0,62.1,60.0,w,INDIVIDUAL,2.0,0.0,0,0,0
3,20000.0,36 months,6.62,614.08,A,A2,_RARE_,9 years,MORTGAGE,110000.0,Source Verified,Fully Paid,debt_consolidation,_RARE_,15.45,8.0,0.0,5275.0,38.2,24.0,f,INDIVIDUAL,2.0,0.0,0,0,0
4,13600.0,36 months,7.29,421.74,A,A4,_RARE_,7 years,MORTGAGE,115000.0,Source Verified,Fully Paid,home_improvement,_RARE_,6.84,9.0,0.0,191.0,0.8,17.0,f,INDIVIDUAL,1.0,0.0,1,0,0


In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 14.0.1+7, mixed mode, sharing)
  Starting server from C:\Users\grego\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\grego\AppData\Local\Temp\tmpw4huq5i4
  JVM stdout: C:\Users\grego\AppData\Local\Temp\tmpw4huq5i4\h2o_grego_started_from_python.out
  JVM stderr: C:\Users\grego\AppData\Local\Temp\tmpw4huq5i4\h2o_grego_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.1
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_grego_54uadn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.947 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [16]:
h2o_train = h2o.H2OFrame(df_train_prepped)
h2o_train['loan_status'] = h2o_train['loan_status'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
predictors = df_train_prepped.columns.to_list()
predictors.remove('loan_status')

In [18]:
# Set up gbm parameter grid and fit in h2o
hyper_params = {
    'ntrees': [100,200],
    'max_depth': [2,4,6],
    'learn_rate': [0.05, 0.1]
}

gbm = H2OGradientBoostingEstimator(
    distribution='bernoulli',
    seed = 1)

h2o_train1, h2o_train2 = h2o_train.split_frame(
    ratios=[0.8],
    seed = 1)

xgb_grid = H2OGridSearch(
    model = gbm,
    hyper_params = hyper_params
)

xgb_grid.train(
    training_frame = h2o_train1,
    x = predictors,
    y = 'loan_status',
    validation_frame = h2o_train2)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [19]:
grid_results = xgb_grid.get_grid(
    sort_by="logloss")

In [20]:
best_params = grid_results.models[0].actual_params
best_params = dict((k,best_params[k]) for k in ('ntrees','max_depth','learn_rate'))

In [21]:
best_params

{'ntrees': 200, 'max_depth': 4, 'learn_rate': 0.1}

In [22]:
df_test_prepped = p.transform(df_test)

h2o_test = h2o.H2OFrame(df_test_prepped)
h2o_test['loan_status'] = h2o_test['loan_status'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
gbm = H2OGradientBoostingEstimator(
    distribution='bernoulli',
    model_id = 'final_gbm',
    **best_params,
    seed = 1)

In [24]:
gbm.train(
training_frame = h2o_train,
    x = predictors,
    y = 'loan_status',
    validation_frame = h2o_test)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [25]:
#path = os.getcwd() + '/lendingclub-app/src/main/resources/'
#if not os.path.exists(path):
#    os.makedirs(path)
#gbm.download_mojo(path=path, get_genmodel_jar=False)

In [26]:
gbm.predict(h2o_test[0:10,:])

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,Charged Off,Fully Paid
Fully Paid,0.195452,0.804548
Fully Paid,0.109756,0.890244
Fully Paid,0.146885,0.853115
Fully Paid,0.534436,0.465564
Fully Paid,0.0735955,0.926404
Fully Paid,0.12821,0.87179
Charged Off,0.576497,0.423503
Fully Paid,0.0650109,0.934989
Fully Paid,0.151643,0.848357
Fully Paid,0.0378637,0.962136


In [29]:
df_test.head(10).to_pickle(path = 'test_cases.pkl')

In [31]:
with open('pipeline.pkl','wb') as f:
    pickle.dump(p,f)

In [30]:
h2o.shutdown()